In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [6]:
INPUT_DIR = '../data/m5_forecasting_accuracy'
calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
selling_prices_df = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')
sample_submission = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')
sales_train_val = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv')
sales_train_eval = pd.read_csv(f'{INPUT_DIR}/sales_train_evaluation.csv')

In [9]:
from data_preprocessing import load_pivoted_dataframes, preprocess_missing_data, prepare_df_with_covariates

pivoted_covariates = load_pivoted_dataframes(sales_train_val)
selling_prices_df = preprocess_missing_data(selling_prices_df)

sales_train_val_transposed1, non_sales_columns_val, sell_price_columns_val, sales_columns_val =\
                    prepare_df_with_covariates(sales_train_val, selling_prices_df, 'validation', calendar, pivoted_covariates)

sales_train_eval_transposed1, non_sales_columns_eval, sell_price_columns_eval, sales_columns_eval =\
                    prepare_df_with_covariates(sales_train_eval, selling_prices_df, 'evaluation', calendar, pivoted_covariates)



Pandas Apply: 100%|██████████| 1941/1941 [00:00<00:00, 2867.28it/s]


In [4]:
from data_preprocessing import darts_preprocessing
DATE_COLUMN = 'date'

train_scaled, val_scaled, train_covariates_scaled, val_covariates_scaled, scaler_series_val, scaler_covariates_val, df_sales, df_covariates, label_encoders_val =\
            darts_preprocessing(sales_train_val_transposed1, None, None,\
                                sales_columns_val, non_sales_columns_val,\
                                sell_price_columns_val, 'validation', None)

eval_scaled, eval_covariates_scaled, scaler_series_eval, scaler_covariates_eval, df_sales, df_covariates, label_encoders_eval =\
            darts_preprocessing(sales_train_eval_transposed1, scaler_series_val, scaler_covariates_val,\
                                sales_columns_eval,non_sales_columns_eval,\
                                sell_price_columns_eval, 'evaluation', label_encoders_val)

In [7]:
import torch
from darts.models import NHiTSModel
from darts.models import NBEATSModel
from darts.dataprocessing.transformers.scaler import Scaler
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from darts.metrics import mae, mse
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)
from pytorch_lightning.callbacks import Callback
from darts.metrics import rmsle
from torchmetrics import MeanSquaredError

# stop training when validation loss does not decrease more than 0.05 (`min_delta`) over
# a period of 5 epochs (`patience`)
my_stopper = EarlyStopping(
    monitor="train_MeanSquaredError",
    patience=20,
    min_delta=0.002,
    mode='min',
)

torch_metrics = MeanSquaredError()
    
# model = NHiTSModel(
#     input_chunk_length=7,  # Example: trying three different input chunk lengths
#     output_chunk_length=28,      # Example: trying two different output chunk lengths
# #     num_blocks=2,             # Number of blocks in the N-HiTS architecture
# #     num_layers=2,                # Number of layers in each block
# #     layer_widths=16,           # Width of each layer    
#     n_epochs=100,
#     torch_metrics=torch_metrics,
#     pl_trainer_kwargs={
#                  "accelerator":"gpu",
#                  "devices": -1,
#                  "callbacks": [my_stopper]    
#                  }        
# )

from darts.models import TCNModel
from darts.models import BlockRNNModel
from darts.models import TransformerModel

model = TransformerModel(
    input_chunk_length=1,
    output_chunk_length=1,
#     model = 'GRU', #(“RNN”, “LSTM” or “GRU”)    
#     hidden_dim=15, 
#     n_rnn_layers=20,    
    n_epochs=1,
    optimizer_cls = torch.optim.Rprop,
    torch_metrics=torch_metrics,
    pl_trainer_kwargs={
                 "accelerator":"gpu",
                 "devices": -1,
                 "callbacks": [my_stopper]    
                 },
    optimizer_kwargs ={
        'lr': 1e-4,       
    }    
)

model.fit(series=train_scaled, past_covariates=train_covariates_scaled,) 

In [ ]:
model.fit(series=train_scaled, past_covariates=train_covariates_scaled,) 